In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
import datetime

# Рабочее пространство



In [2]:
#Очистка Датафрейма
def ClearDF(df, parameters):
    return df[parameters]

In [3]:
#Переименование параметров
def renameDF(df, parameters, rename_parameters):
    for i in range(len(parameters)):
        df.rename(columns={parameters[i]: rename_parameters[i]}, inplace=True)
    return df

In [4]:
#Группировка параметров по дате
def groupDF(df, group_parameter):
    df_copy = df.copy()
    df_copy[group_parameter] = pd.to_datetime(df[group_parameter]).dt.date
    df_copy = df_copy.groupby(group_parameter).sum()
    return df_copy

In [5]:
#Cоздание ДатаФрейма для группировки данных по интервалам дня(утро, день, вечер, ночь)
def createDfinterval(intervals, parameters):
    df = pd.DataFrame({'interval': intervals})
    #Заполнение ДатаФрейма нулями
    for i in range(len(parameters)):
        df[parameters[i]] = [0] * 5
    return df

In [6]:
#Группировка параметров по времени суток
def paddingDF(df, df_xlsx, date, parameters):
    for i in range(df_xlsx.shape[0]):
        #Получение чсисла равное часам в данной записи
        hour = df_xlsx[date][i].hour
        if((hour>= 0 and hour < 6) or (hour>= 22 and hour < 24)) :
            for j in range(len(parameters)):
                df[parameters[j]][0] += df_xlsx[parameters[j]][i]                                                                                               
        if(hour>= 6 and hour < 10):
            for j in range(len(parameters)):
                df[parameters[j]][1] += df_xlsx[parameters[j]][i]
        if(hour>= 10 and hour < 14):
            for j in range(len(parameters)):
                df[parameters[j]][2] += df_xlsx[parameters[j]][i]
        if(hour>= 14 and hour < 18):
            for j in range(len(parameters)):
                df[parameters[j]][3] += df_xlsx[parameters[j]][i]
        if(hour>= 18 and hour < 22):
            for j in range(len(parameters)):
                df[parameters[j]][4] += df_xlsx[parameters[j]][i]
        return df

In [7]:
#Группировка по признаку
def groupDFcsv(df, group_parameter, parameters):
    df_parameter = ClearDF(df, parameters)
    df_parameter = df_parameter.groupby(group_parameter).sum()
    return df_parameter

In [8]:
#ТГ
def tg_full():
    #Создание ДатаФраймов 
    dep_xlsx_tg = pd.read_excel("jupyter/dep_tg.xlsx")
    
    #Исследуемые параметры
    parameters_tg = ["Views","Text","VR","Date","Media 1"]
        
    #Название параметров
    rename_parameters_tg = ['views','title','coef_ohv','datetime_post','media']
    
    #Исследуемые параметры
    investigated_parameters_tg = ['datetime_post','views','coef_ohv']
    
    #Исследуемые параметры без учета времени
    investigated_parameters_tg_group = ['views','coef_ohv']
    
    #Переменная времени(индекс)
    date = 'datetime_post'
    
    #Интервалы времени
    interval = ['night', 'morning', 'snack', 'lunch', 'dinner']
    
    
    #Вызов функции очистки
    dep_xlsx_tg = ClearDF(dep_xlsx_tg, parameters_tg)
    
    #Вызов функции переименовывания параметров
    dep_xlsx_tg = renameDF(dep_xlsx_tg, parameters_tg, rename_parameters_tg)
    
    #Вызов функции группировки по параметру datetime_post
    dep_xlsx_tg_date = groupDF(dep_xlsx_tg,'datetime_post')
    
    #Фызуов функции создания Датафрейма для группировки по интервалам
    dep_xlsx_tg_date_interval = createDfinterval(interval, investigated_parameters_tg_group)
    
    #Фузов функции заполнения ДатаФрейма
    dep_xlsx_tg_date_interval = paddingDF(dep_xlsx_tg_date_interval, dep_xlsx_tg, date, investigated_parameters_tg_group)
    
    index_xlsx = dep_xlsx_tg_date.index[:]
    dep_xlsx_tg_date["datetime_post"] = index_xlsx
    dep_xlsx_tg_date.index = range(dep_xlsx_tg_date.shape[0])
    dep_xlsx_tg_date.index.name = ''
    
    return dep_xlsx_tg_date, dep_xlsx_tg_date_interval

In [9]:
#ВК
def vk_full():
    #Создание ДатаФраймов 
    dep_xlsx_vk = pd.read_excel("jupyter/dep_vk.xlsx")
    
    #Исследуемые параметры
    parameters_vk = ["Likes","Reposts","Comments","Views","ER","Text","VR","Date","Media 1"]
    
    #Название параметров
    rename_parameters_vk = ['likes','reposts','comments','views','coef_vov','title','coef_ohv','datetime_post','media']
    
    #Исследуемые параметры
    investigated_parameters_vk = ['datetime_post','likes','comments','views','reposts','coef_vov','coef_ohv']
    
    #Исследуемые параметры без учета времени
    investigated_parameters_vk_group = ['likes','comments','views','reposts','coef_vov','coef_ohv']
    
    #Переменная времени(индекс)
    date = 'datetime_post'
    
    #Интервалы времени
    interval = ['night', 'morning', 'snack', 'lunch', 'dinner']
    
    
    #Вызов функции очистки
    dep_xlsx_vk = ClearDF(dep_xlsx_vk, parameters_vk)
    
    #Вызов функции переименовывания параметров
    dep_xlsx_vk = renameDF(dep_xlsx_vk, parameters_vk, rename_parameters_vk)
    
    #Вызов функции группировки по параметру datetime_post
    dep_xlsx_vk_date = groupDF(dep_xlsx_vk,'datetime_post')
    
    #Фызуов функции создания Датафрейма для группировки по интервалам
    dep_xlsx_vk_date_interval = createDfinterval(interval, investigated_parameters_vk_group)
    
    #Фузов функции заполнения ДатаФрейма
    dep_xlsx_vk_date_interval = paddingDF(dep_xlsx_vk_date_interval, dep_xlsx_vk, date, investigated_parameters_vk_group)
    
    index_xlsx = dep_xlsx_vk_date.index[:]
    dep_xlsx_vk_date["datetime_post"] = index_xlsx
    dep_xlsx_vk_date.index = range(dep_xlsx_vk_date.shape[0])
    dep_xlsx_vk_date.index.name = ''
    
    return dep_xlsx_vk_date, dep_xlsx_vk_date_interval

In [10]:
#CSV
def csv_full():
    #Создание ДатаФраймов 
    dep_csv = pd.read_csv("jupyter/dep_user.csv")
    
    #Исследуемые параметры
    parameters_csv_gender = ['gender', 'contract_sum', "purchase_sum", "purchase_count"]
    parameters_csv_city = ['city', 'contract_sum', "purchase_sum", "purchase_count"]
    parameters_csv_birth_date = ['birth_date', 'contract_sum', "purchase_sum", "purchase_count"]
    
    #Вызов функций для группировки по признаку
    #Группировка по гендеру
    dep_csv_gender = groupDFcsv(dep_csv, 'gender',parameters_csv_gender)
    #Группировка по городу
    dep_csv_city = groupDFcsv(dep_csv, 'city',parameters_csv_city)
    #Группировка по дате рождения
    dep_csv_birth_date = groupDFcsv(dep_csv, 'birth_date', parameters_csv_birth_date)
    
    index_xlsx = dep_csv_gender.index[:]
    dep_csv_gender["gender"] = index_xlsx
    dep_csv_gender.index = range(dep_csv_gender.shape[0])
    dep_csv_gender.index.name = ''
    
    index_xlsx = dep_csv_birth_date.index[:]
    dep_csv_birth_date["city"] = index_xlsx
    dep_csv_birth_date.index = range(dep_csv_birth_date.shape[0])
    dep_csv_birth_date.index.name = ''
    
    
    index_xlsx = dep_xlsx_vk_date.index[:]
    dep_csv_birth_date["birth_date"] = index_xlsx
    dep_csv_birth_date.index = range(dep_csv_birth_date.shape[0])
    dep_csv_birth_date.index.name = ''
        
    return dep_csv_gender, dep_csv_city, dep_csv_birth_date

In [11]:
df1, df2 = tg_full()

C:\Users\Ssonder\AppData\Local\Temp\ipykernel_10116\3908740193.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[parameters[j]][3] += df_xlsx[parameters[j]][i]


In [12]:
df1

,views,coef_ohv,datetime_post
,,,
0,3232,66.625,2022-05-12
1,3344,68.934,2022-05-13
2,3634,74.912,2022-05-14
3,3422,70.542,2022-05-16
4,7019,144.692,2022-05-17
5,2959,60.998,2022-05-19
6,7142,147.227,2022-05-20
7,3974,81.921,2022-05-21
8,2974,61.307,2022-05-24


In [20]:
def parseData(data):
    if type(data) == np.int64:
        return int(data)
    if type(data) == datetime.date:
        return data.isoformat()
    return data

    
def arrXY(df, dfXName,dfYname,xKey,yKey):
    resultArray = []
    for i in range(df.shape[0]):        
        resultArray.append({xKey: parseData(df[dfXName][i]), yKey: parseData(df[dfYname][i])})
    return resultArray


def series_arr(df, dfXName,dfYname,xKey,yKey):
    resultArray = []
    for i in range(df.shape[0]):
        resultArray.append({'data':[{xKey: parseData(df[dfXName][i]), yKey: parseData(df[dfYname][i])}],'yName': dfYname })
    return resultArray

C:\Users\Ssonder\AppData\Local\Temp\ipykernel_10116\3908740193.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[parameters[j]][2] += df_xlsx[parameters[j]][i]


[{'data': [[{'x': datetime.date(2022, 5, 12), 'y': 0}, {'x': datetime.date(2022, 5, 13), 'y': 8}, {'x': datetime.date(2022, 5, 14), 'y': 6}, {'x': datetime.date(2022, 5, 16), 'y': 0}, {'x': datetime.date(2022, 5, 17), 'y': 0}, {'x': datetime.date(2022, 5, 18), 'y': 6}, {'x': datetime.date(2022, 5, 19), 'y': 0}, {'x': datetime.date(2022, 5, 20), 'y': 1}, {'x': datetime.date(2022, 5, 21), 'y': 1}, {'x': datetime.date(2022, 5, 22), 'y': 0}, {'x': datetime.date(2022, 5, 23), 'y': 0}, {'x': datetime.date(2022, 5, 24), 'y': 0}, {'x': datetime.date(2022, 5, 25), 'y': 2}, {'x': datetime.date(2022, 5, 26), 'y': 0}, {'x': datetime.date(2022, 5, 27), 'y': 0}, {'x': datetime.date(2022, 5, 28), 'y': 2}, {'x': datetime.date(2022, 5, 30), 'y': 4}, {'x': datetime.date(2022, 5, 31), 'y': 0}, {'x': datetime.date(2022, 6, 1), 'y': 3}, {'x': datetime.date(2022, 6, 2), 'y': 0}, {'x': datetime.date(2022, 6, 3), 'y': 0}, {'x': datetime.date(2022, 6, 4), 'y': 3}, {'x': datetime.date(2022, 6, 6), 'y': 1}, {'x'